<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Figure 1: Observed and modelled forest risks

_Authors: Oriana Chegwidden and Jeremy Freeman_

The methods below conduct the analyses to recreate Figure 1 included the
manuscript <<< insert doi >>>. They draw from a model data produced by running
`../../../scripts/fire.py` to create a the file located at
`az:carbonplan-scratch/data/fire.zarr`.


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forests import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forests.utils import get_store
import altair as alt
import rioxarray
from carbonplan.data import cat
alt.data_transformers.disable_max_rows()

## Load in the fire data


In [ ]:
coarsen = 4
store = "az"
tlim = ("1984", "2018")

#### Historical simulation


In [ ]:
historical_ds = xr.open_zarr(
    get_store("carbonplan-scratch", "data/fire_historical_v2.zarr")
)

#### Observations


In [ ]:
mask = (
    load.nlcd(store=store, year=2001).sel(band=[41, 42, 43, 90]).sum("band")
    > 0.25
).astype("float")
mtbs = load.mtbs(store=store, coarsen=coarsen, tlim=tlim, mask=mask)
mtbs = mtbs.assign_coords({"x": historical_ds.x, "y": historical_ds.y})

## Load in the drought data


#### Observations


#### Historical simulation


In [ ]:
target = cat.nlcd.raster.to_dask()
source = xr.open_rasterio(
    "https://carbonplan.blob.core.windows.net/carbonplan-scratch/from_bill/Fig1D_AllMortModel_ModeledFIAlongHistMort_3-19-2021.tiff"
)
source = source.where(source > -1)
drought = source.rio.reproject_match(target)
drought = (
    drought.where(drought > -1)
    .coarsen(x=coarsen, y=coarsen, boundary="trim")
    .mean()
    .sel(band=1)
)
drought = drought.assign_coords({"lat": mtbs.lat, "lon": mtbs.lon})

In [ ]:
mtbs

## Load in the biotic agents data


#### Observations


#### Historical simulation


## Plot the figure for all three impacts


In [ ]:
full_plot = alt.vconcat().resolve_scale(color="independent")
fire_chart = plot.fire.simple_map(
    mtbs["monthly"].groupby("time.year").sum().mean(dim="year"),
    data2=historical_ds["historical"]
    .groupby("time.year")
    .sum()
    .mean(dim="year"),
    title1="Observed",
    title2="Modelled",
    clim=(0.001, 0.008),
    clabel="Burn area (fraction/year)",
    cmap=["white", "#EA9755"],
)
insects_chart = plot.fire.simple_map(
    mtbs["monthly"].groupby("time.year").sum().mean(dim="year"),
    data2=historical_ds["historical"]
    .groupby("time.year")
    .sum()
    .mean(dim="year"),
    title1="Observed",
    title2="Modelled",
    clim=(0.001, 0.008),
    clabel="Biotic agents morality ()",
    cmap=["white", "#85A2F7"],
)
drought_chart = plot.fire.simple_map(
    mtbs["monthly"].groupby("time.year").sum().mean(dim="year"),
    data2=drought,
    title1="Observed",
    title2="Modelled",
    clim=(0.001, 0.02),
    clabel="Drought mortality ()",
    cmap=["white", "#E587B6"],
)

full_plot &= fire_chart
full_plot &= insects_chart
full_plot &= drought_chart

In [ ]:
full_plot.configure_view(strokeOpacity=0)

In [ ]:
plot.fire.monthly(
    mtbs,
    "monthly",
    clim=(0.0001, 0.002),
    cmap="reds",
    clabel="Burn area [fraction/month]",
)

In [ ]:
plot.fire.monthly(
    historical_ds,
    "historical",
    clim=(0.0001, 0.002),
    cmap="reds",
    clabel="Burn area [fraction/month]",
)